In [1]:
!pip install langchain transformers torch


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline  # Updated Import

# ✅ Load GPT-2 Model & Tokenizer
model_name = "gpt2"  # Use "gpt2-medium" or "gpt2-large" for better performance
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# ✅ Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Create a Text-Generation Pipeline with Fixes
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100,  # Ensure it doesn’t generate too much
    temperature=0.7,  # Controls randomness
    repetition_penalty=1.2,  # ✅ Reduce repetition
    pad_token_id=tokenizer.eos_token_id,  # Avoid padding errors
    truncation=True  # ✅ Explicitly enable truncation
)

# ✅ Wrap in LangChain's LLM Interface
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# ✅ Conversation Memory (Limited to Last 5 Exchanges)
conversation_history = []

# ✅ Function to Chat in Jupyter
def chat_gpt2(user_input):
    global conversation_history  # Store chat history across multiple cells
    
    if user_input.lower() == "exit":
        return "Chatbot: Goodbye! 👋"
    
    # ✅ Keep only the last 5 exchanges (prevents long repetitions)
    if len(conversation_history) > 5:
        conversation_history.pop(0)
    
    # ✅ Format the conversation
    conversation_history.append(f"User: {user_input}\nAI:")
    formatted_conversation = "\n".join(conversation_history)
    
    # ✅ Generate response
    response = llm(formatted_conversation)
    
    # ✅ Extract AI's response (removes "User:" and "AI:" from history)
    ai_response = response.strip().split("AI:")[-1].strip()
    
    # ✅ Add AI response to history
    conversation_history.append(f"AI: {ai_response}")
    
    return f"Chatbot: {ai_response}"


Device set to use cpu


In [2]:
chat_gpt2("Tell me a joke.")


C:\Users\student\AppData\Local\Temp\ipykernel_8844\4238874717.py:48: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(formatted_conversation)


"Chatbot: Oh, that's awesome! I really like it though... It would have been great if you wrote something in our story about how your parents were kind of the only people who could take care (laughing) because they didn't want to hear any jokes at all when we're making this movie as well.\n\n[Laughter] Well my name is Jonathan and here are some cool things out there for us from what everyone can see right now so"

In [3]:
chat_gpt2("exit")


'Chatbot: Goodbye! 👋'